In [1]:
print("Utviklingsmiljøet er klart!")

Utviklingsmiljøet er klart!


# Beskrivelse av værdata-innhenting med Frost og OpenCage API

## Introduksjon
Dette Python-skriptet henter historiske værdata (temperatur, nedbør, vindhastighet, luftfuktighet) for en valgt lokasjon i Norge.  
Det bruker API-tjenestene **Frost** (fra Meteorologisk Institutt) og **OpenCage** (for geokoding).

---

## Funksjoner

- **Geokoding**:  
  Bruker OpenCage API til å finne bredde- og lengdegrader for en brukerinndata-lokasjon (f.eks. Bergen, Oslo).
  
- **Finn nærmeste værstasjoner**:  
  Henter alle værstasjoner fra Frost API og beregner avstanden fra brukerens lokasjon til hver stasjon.
  
- **Datainnhenting**:  
  For hver værparameter (element) forsøker skriptet å hente observasjoner fra de nærmeste stasjonene.

- **Databehandling**:  
  Samler dataene i en Pandas DataFrame, fjerner unødvendige kolonner, og lagrer datasett for hvert element til separate CSV-filer.

---

## Brukte bibliotek

- `requests` — for HTTP-kall til API-er
- `pandas` — for å organisere og lagre data
- `os`, `sys` — for filstier og systemmanipulering
- `datetime`, `timedelta` — for håndtering av datoer
- `opencage.geocoder` — for geokoding av lokasjon
- `geopy.distance` — for å beregne geografiske avstander
- Egendefinert modul: `API_Key_Manager_class` for håndtering av API-nøkler

---

## Viktige steg i koden

### 1. Importer nødvendige bibliotek og klasser
Laster inn biblioteker og en egendefinert klasse for API-nøkkelstyring.

### 2. Hent API-nøkler
Henter nøkler til Frost og OpenCage gjennom `APIKeyManager`.

### 3. Geokoding av brukerinput
Brukeren skriver inn en by/sted, og koordinatene hentes via OpenCage API.

### 4. Finn værstasjoner
- Laster ned alle stasjoner fra Frost.
- Filtrerer ut koordinatene.
- Beregner avstanden fra brukerens sted til alle stasjoner.

### 5. Hent observasjonsdata
- Definerer hvilke værparametere som skal hentes (temperatur, nedbør, vind, luftfuktighet).
- For hvert parameter forsøkes datahenting fra de nærmeste stasjonene.

### 6. Databehandling
- Samler observasjonene i en Pandas DataFrame.
- Konverterer tidspunkt til `datetime`-format.
- Rydder opp i overflødige kolonner.

### 7. Lagring
- Filtrerer ut data for hver værparameter.
- Lagrer dataene i fire separate `.csv`-filer.

---

## CSV-filer som genereres

- `temperature_data.csv`
- `precipitation_data.csv`
- `wind_speed_data.csv`
- `humidity_data.csv`

Hver fil inneholder historiske observasjoner for valgt lokasjon de siste 7 årene.

---

## Forutsetninger

- Du må ha installert nødvendige Python-biblioteker (`requests`, `pandas`, `geopy`, `opencage`).

---

# Oppsummering
Dette skriptet automatiserer en omfattende prosess fra **lokasjonssøk** til **datainnsamling**, **databehandling**, og **datapreparering**. Det er spesielt nyttig for prosjekter innen datavitenskap, væranalyse eller maskinlæring.

---

In [1]:
import requests
import pandas as pd
import os
import sys

from datetime import datetime
from datetime import timedelta
from opencage.geocoder import OpenCageGeocode
from geopy.distance import geodesic

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))
from API_Key_Manager_class import APIKeyManager

api_key_manager = APIKeyManager()

# Hent Frost API-nøkkelen direkte fra klassen
client_id = api_key_manager.get_api_key("Frost")

# Hent Opencage API-nøkkelen direkte fra klassen
api_key = api_key_manager.get_api_key("Opencage")

# Sett opp OpenCageGeocode med API-nøkkelen din
geolocator = OpenCageGeocode(api_key)

# Henter inn lokasjon fra brukeren
location_input = input("Enter the location (e.g., Bergen, Oslo, Trondheim, Stavanger, Tromsø): ")

# Geokoder lokasjonen for å få koordinatene
location = geolocator.geocode(location_input)

# Sjekker om geokodingen var vellykket
if location:
    # Henter ut koordinatene
    location_coords = (location[0]['geometry']['lat'], location[0]['geometry']['lng'])
else:
    raise ValueError("Could not find coordinates for the location. Please check the input.")

# Definerer endepunkt og parametere for stasjoner
stations_endpoint = 'https://frost.met.no/sources/v0.jsonld'
stations_parameters = {
    'types': 'SensorSystem'
}

# Henter ut HTTP-responsen og lagrer den som JSON
r = requests.get(stations_endpoint, stations_parameters, auth=(client_id, ''))
stations_json = r.json()

# Henter ut stasjonsinformasjonen
stations = stations_json['data']

# Lager et dictionary med stasjons-ID som nøkkel og koordinater som verdi
weather_stations_coords = {station['id']: (station['geometry']['coordinates'][1], 
station['geometry']['coordinates'][0]) for station in stations if 'geometry' in station}

# Regner ut avstanden til værstasjonene fra lokasjonen
distances = {station: geodesic(location_coords, coords).km for station, coords in weather_stations_coords.items()}

# Sorterer værstasjonene etter avstand til lokasjonen
sorted_stations = sorted(distances, key=distances.get)

# Henter ut dagens dato og datoen x dager tilbake
now = datetime.now()
days = 365*7
offset = now - timedelta(days)
current_date = now.strftime("%Y-%m-%d")
historic_date = offset.strftime("%Y-%m-%d")

# Definerer endepunkt og parametere for observasjoner
observations_endpoint = 'https://frost.met.no/observations/v0.jsonld'

def fetch_data_by_element(stations, elements, max_attempts = 15):
    """
    Fetch data for each element (e.g., temperature, precipitation, wind speed) from the nearest weather stations.
    Tries up to max_attempts stations for each element and returns the data for the first successful attempt.
    """
    all_data = {}
    
    for element in elements:
        print(f"Henter data for element: {element}")
        for attempt in range(min(max_attempts, len(stations))):
            parameters = {
                'sources': stations[attempt],
                'elements': element,
                'referencetime': f'{historic_date}/{current_date}',
            }
            r = requests.get(observations_endpoint, parameters, auth=(client_id, ''))
            json = r.json()

            if r.status_code == 200 and 'data' in json:
                data = json['data']
                if data:
                    print(f"Data retrieved for {element} from station {stations[attempt]}!")
                    all_data[element] = data
                    break
            else:
                print(f"Error retrieving {element} data from station {stations[attempt]}. Trying next station...")
        
        # Hvis ingen data blir funnet for elementet, logges en melding
        if element not in all_data:
            print(f"No data found for {element} from any of the attempted stations.")
    
    return all_data
# Elementene vi ønsker å hente data for
elements = [
    'mean(air_temperature P1D)', 
    'sum(precipitation_amount P1D)', 
    'mean(wind_speed P1D)',
    'mean(relative_humidity P1D)'
]

# Henter data for lokasjonen
data = fetch_data_by_element(sorted_stations, elements)

# Samler de viktige dataene i en DataFrame
df_list = []

for element, data_element in data.items():
    for item in data_element:
        row = pd.DataFrame(item['observations'])
        row['referenceTime'] = item['referenceTime']
        row['sourceId'] = item['sourceId']
        row['elementId'] = element
        df_list.append(row)

# Effektivt å koble sammen alle DataFrames samtidig
df = pd.concat(df_list, ignore_index=True)

df['referenceTime'] = pd.to_datetime(df['referenceTime'])

# Fjerner spesifike kolonner
columns_to_delete = ["level","timeResolution","timeSeriesId","performanceCategory","exposureCategory","qualityCode"]
try:
    df.drop(columns=columns_to_delete, inplace=True)
except Exception as e:
    print("Error:", e)

# Lagrer dataene i CSV-filer (overskriver eksisterende fil)

# Filtrerer og lagrer data for temperatur
temperature_data = df[df['elementId'] == 'mean(air_temperature P1D)']
temperature_data.to_csv("../data/temperature_data.csv", index=False)

# Filtrerer og lagrer data for nedbør
precipitation_data = df[df['elementId'] == 'sum(precipitation_amount P1D)']
precipitation_data.to_csv("../data/precipitation_data.csv", index=False)

# Filtrerer og lagrer data for vindhastighet
wind_speed_data = df[df['elementId'] == 'mean(wind_speed P1D)']
wind_speed_data.to_csv("../data/wind_speed_data.csv", index=False)

# Filtrerer og lagrer data for relativ luftfuktighet
humidity_data = df[df['elementId'] == 'mean(relative_humidity P1D)']
humidity_data.to_csv("../data/humidity_data.csv", index=False)


Henter data for element: mean(air_temperature P1D)
Data retrieved for mean(air_temperature P1D) from station SN50570!
Henter data for element: sum(precipitation_amount P1D)
Data retrieved for sum(precipitation_amount P1D) from station SN50570!
Henter data for element: mean(wind_speed P1D)
Data retrieved for mean(wind_speed P1D) from station SN50570!
Henter data for element: mean(relative_humidity P1D)
Data retrieved for mean(relative_humidity P1D) from station SN50570!
